In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
import re
import xml.etree.ElementTree as ET

In [34]:
tropnames = {'etnakhta': u'\u0591', 'segol': u'\u0592', 'shalshelet': u'\u0593', 'katan': u'\u0594', 'gadol': u'\u0595',
             'tipkha': u'\u0596', 'revii': u'\u0597', 'tsinnorit': u'\u0598', 'pashta': u'\u0599', 'yetiv': u'\u059a', 'tevir': u'\u059b',
             'geresh': u'\u059c', 'gereshmukdam': u'\u059d', 'gershayim': u'\u059e', 'karnepara': u'\u059f', 'telishagedola': u'\u05a0',
             'pazer': u'\u05a1', 'munakh': u'\u05a3', 'mapakh': u'\u05a4', 'merkha': u'\u05a5',
             'merkhakfula': u'\u05a6', 'darga': u'\u05a7', 'kadma': u'\u05a8', 'telishaketana': u'\u05a9', 'yerakhbenyomo': u'\u05aa',
             'sofpasuk': u'\u05c3', 'zarka': u'\u05ae'}

# per wikipedia: Note that both marks have been wrongly named by Unicode.[5][6] Zarqa/tsinnor corresponds to Unicode
# "Hebrew accent zinor", code point U+05AE (where "zinor" is a misspelled form for tsinnor), while tsinnorit maps to
# "Hebrew accent zarqa", code point U+0598. 

tropchars = {tropnames[t]: t for t in tropnames}

In [53]:
sfarim = ['bereshit', 'shmot', 'vayikra', 'bmidbar', 'dvarim']

counts = {}
for sefer in sfarim:
    counts[sefer] = {}
    tree = ET.parse(sefer + '.xml')
    root = tree.getroot() 
    prakim = root.findall('.//c')
    for perek in prakim:
        pereknum = int(perek.attrib['n'])
        if pereknum not in counts[sefer]: counts[sefer][pereknum] = {}
        psukim = perek.findall('v')
        for pasuk in psukim:
            pasuknum = int(pasuk.attrib['n'])
            if pasuknum not in counts[sefer][pereknum]:
                row = {name: 0 for name in tropnames.keys()}
                row['sefer'] = sefer
                row['pasuk'] = pasuknum
                row['perek'] = pereknum
                counts[sefer][pereknum][pasuknum] = row
            words = pasuk.findall('w')
#             print words.map(text)
        
#             if 'wordcount' not in counts[sefer][pereknum][pasuknum]:
#                 counts[sefer][pereknum][pasuknum]['wordcount'] = len(words)
#             else:
#                 counts[sefer][pereknum][pasuknum]['wordcount'] += len(words)
#             for wordobj in words:
#                 word = wordobj.text
#                 for trop in tropnames:
#                     if tropnames[trop] in word: # if there is ever the same trop more than once on one word, it won't be captured here
#                         counts[sefer][pereknum][pasuknum][trop] += 1

In [101]:
wordslist = map(lambda w: list(w.text), words)

In [137]:
troplist = map(lambda w: filter(lambda c: c in tropchars, w), wordslist)
troplist = filter(lambda w: len(w) > 0, troplist)

In [134]:
def commajoin(w):
    return ','.join(w)

In [143]:
troplist[0].append(u'\u0596') # for debug purposes

In [147]:
troplist

[[u'\u0599', u'\u0596', u'\u0596'],
 [u'\u05a3'],
 [u'\u0594'],
 [u'\u0596'],
 [u'\u05a3'],
 [u'\u0591'],
 [u'\u0599'],
 [u'\u05a3'],
 [u'\u0594'],
 [u'\u0596'],
 [u'\u05c3']]

In [150]:
# there's probably a better way to do this, but I can't think of it
flattroplist = []
for w in troplist:
    for c in w:
        flattroplist.append(c)

In [154]:
commanjoin(flattroplist)

u'\u0599,\u0596,\u0596,\u05a3,\u0594,\u0596,\u05a3,\u0591,\u0599,\u05a3,\u0594,\u0596,\u05c3'

In [141]:
pasuktropstring = map(commajoin, troplist)

In [142]:
pasuktropstring
# [tropchars[s] for s in pasuktropstring]

[u'\u0599,\u0596',
 u'\u05a3',
 u'\u0594',
 u'\u0596',
 u'\u05a3',
 u'\u0591',
 u'\u0599',
 u'\u05a3',
 u'\u0594',
 u'\u0596',
 u'\u05c3']

In [155]:
map(lambda c: tropchars[c], flattroplist)

['pashta',
 'tipkha',
 'tipkha',
 'munakh',
 'katan',
 'tipkha',
 'munakh',
 'etnakhta',
 'pashta',
 'munakh',
 'katan',
 'tipkha',
 'sofpasuk']

In [21]:
tropnames.values()

[u'\u0597',
 u'\u059d',
 u'\u05a6',
 u'\u059e',
 u'\u059b',
 u'\u059f',
 u'\u0595',
 u'\u05a0',
 u'\u0599',
 u'\u0593',
 u'\u0596',
 u'\u059a',
 u'\u0592',
 u'\u05aa',
 u'\u05ae',
 u'\u05a3',
 u'\u05a5',
 u'\u05a8',
 u'\u0591',
 u'\u05c3',
 u'\u0598',
 u'\u059c',
 u'\u05a9',
 u'\u05a7',
 u'\u05a1',
 u'\u05a4',
 u'\u0594']